In [1]:
import os
import sys
import pickle
sys.path.append(f"{os.environ['MINERVA_HOME']}/code")
from mil_model import MILModel
from mil_dataset import MILTwitterDataset
from train_mil import MILTrainingArguments
from transformers import AutoTokenizer
import torch
from tqdm import tqdm
import jsonlines
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
results_dir = f"{os.environ['MINERVA_HOME']}/models/test/ratio_0.1"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Training arguments for consistency
# Need to import MILTrainingArguments to work
training_args = torch.load(f"{results_dir}/training_args.bin")

Load trained model and dataset for evaluation

In [3]:
model = MILModel().from_pretrained(results_dir)
tokenizer = AutoTokenizer.from_pretrained(training_args.instance_model)

dataset = MILTwitterDataset(
    f"{training_args.dataset_dir}/test.jsonl",
    tokenizer,
    samples_per_bag=training_args.num_tweets_per_day,
    random_seed=training_args.seed
)

dataloader = torch.utils.data.DataLoader(
    dataset, 
    shuffle=False, 
    batch_size=20, 
    collate_fn=dataset.collate_function
)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

## Run inference

In [5]:
output_file = f"{results_dir}/test_predictions.json"
model = model.eval().to(device)
with jsonlines.open(output_file, "w") as f:
    with torch.inference_mode():
        for i, batch in enumerate(tqdm(dataloader, ncols=0)):
            # Run inference
            batch = batch.to(device)
            output = model(**batch)

            # Print the key instances
            for filename, label, ids, text, prob, key_ids, tweet_probs in zip(batch["bag_id"], batch["labels"], batch["instance_ids"], batch["instance_text"], output.logits, output.key_instances, output.instance_probs):
                row = {
                    "filename": filename,
                    "probability": prob.item(),
                    "prediction": 1 if prob.item() > 0.5 else 0,
                    "label": int(label.item()),
                    "key_tweet_ids": key_ids,
                    "key_tweets": [text[ids.index(t)] for t in key_ids],
                    # "key_tweet_probs": list(tweet_probs),
                }
                f.write(row)


  0% 0/6 [00:01<?, ?it/s]


AttributeError: 'dict' object has no attribute 'to'